In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
pd.set_option('display.max_rows',10000)
pd.set_option('display.max_columns',100)



In [ ]:
df=pd.read_csv('/kaggle/input/ecommerce-data/data.csv',encoding='ISO-8859-1')
df.head()

In [ ]:
df.rename(index=str,columns={'InvoiceNo': 'invoice_num',
                            'StockCode': 'stock_code',
                            'Description': 'description',
                            'Quantity': 'quantity',
                            'InvoiceDate': 'invoice_date',
                            'UnitPrice': 'unit_price',
                            'CustomerID': 'cust_id',
                            'Country':'country'},inplace=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
#check out the rows with missing values
df[df.isnull().any(axis=1)].head()

In [ ]:
#change the invoice_date format-string to timestamp format
df['invoice_date']=pd.to_datetime(df['invoice_date'],format='%m/%d/%Y %H:%M')

In [ ]:
df.tail()

In [ ]:
df['description']=df['description'].str.lower()

In [ ]:
df.head()

In [ ]:
df_new=df.dropna()
df_new.isnull().sum().sort_values(ascending=False)

In [ ]:
df_new['cust_id']=df_new['cust_id'].astype('int64')
df_new.head()

In [ ]:
df_new.info()

In [ ]:
df_new.groupby(by=['cust_id','country'],as_index=False)['invoice_num'].count()

In [ ]:
df_new['quantity'].describe()

In [ ]:
df_new=df_new[df_new['quantity']>0]
df_new['quantity'].describe()

In [ ]:
df_new['amount']=df_new['quantity']*df_new['unit_price']
df_new.head()

In [ ]:
df_new[df_new['cust_id'].isnull()].head()

In [ ]:
df_new['cancelled']=np.where(df_new['invoice_num'].apply(lambda x:x[0]=='C'),1,0)
df_new[df_new['cancelled']==0]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()#seaborn defaults
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
#which stock code is most common
stockcode_counts=df_new['stock_code'].value_counts().sort_values(ascending=False)
fig,ax=plt.subplots(2,1,figsize=(20,15))
sns.barplot(stockcode_counts.iloc[0:20].index,
            stockcode_counts.iloc[0:20].values,
           ax=ax[0],palette="Oranges_r")
ax[0].set_ylabel('counts')
ax[0].set_xlabel('stockcode')
ax[0].set_title('stockcode distribution')
sns.distplot(np.round(stockcode_counts/df_new.shape[0]*100,2),
           kde=False,
           bins=20,
           ax=ax[1],color='Orange')
ax[1].set_title('% of stockcode distribution')
ax[1].set_xlabel('% of invoice with this code')
ax[1].set_ylabel('Frequency')


In [ ]:
print(list(ax))

In [ ]:
def count_numeric_chars(l):
    return sum(1 for c in l if c.isdigit())

In [ ]:
df_new['stockcodelength']=df_new['stock_code'].apply(lambda l: len(l))
df_new['stockcodenumeric']=df_new['stock_code'].apply(lambda l: count_numeric_chars(l))

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(20,5))
sns.countplot(df_new['stockcodelength'],palette='Oranges_r',ax=ax[0])
sns.countplot(df_new['stockcodenumeric'],palette='Oranges_r',ax=ax[1])
ax[0].set_xlabel('Length of stockcode')
ax[1].set_xlabel('numeric count in stockcode')

In [ ]:
df_new.loc[df_new['description'].isnull()==False,'lower_description']=df_new.loc[df_new['description'].isnull()==False,'description'].apply(lambda l:l.lower())

In [ ]:
df_new.head()

In [ ]:
#is there any missing value left
df_new.isnull().sum().sum()

In [ ]:
df_new.loc[df_new['stockcodenumeric']<5]['lower_description'].value_counts()

In [ ]:
data=df_new.loc[(df_new['stockcodenumeric']==5)&(df_new['stockcodelength'])].copy()

In [ ]:
data.head()

In [ ]:
data['stock_code'].nunique()

In [ ]:
data.drop(['stockcodelength','stockcodenumeric'],axis=1)

In [ ]:
description_counts=data['description'].value_counts().sort_values(ascending=False).iloc[0:30]
plt.figure(figsize=(20,5))
sns.barplot(description_counts.index,description_counts.values,palette='Purples_r')
plt.ylabel('Counts')
plt.title('which product descriptions are most commom?')
plt.xticks(rotation=90)


In [ ]:
def count_lower_chars(l):
    return sum(1 for c in l if c.islower())

In [ ]:
data['descriptionlength']=data['description'].apply(lambda l: len(l))
data['LowCharsInDescription']=data['description'].apply(lambda l: count_lower_chars(l))

In [ ]:
fig, ax=plt.subplots(1,2,figsize=(20,5))
sns.countplot(data['descriptionlength'],ax=ax[0],color='Purple')
sns.countplot(data['LowCharsInDescription'],ax=ax[1],color='Purple')
ax[1].set_yscale('log')

In [ ]:
lowchar_counts=data.loc[data['LowCharsInDescription']>0]['description'].value_counts()[0:10]
plt.figure(figsize=(15,3))
sns.barplot(lowchar_counts.index, lowchar_counts.values, palette="Purples_r")
plt.xticks(rotation=90);

In [ ]:
lowchar_counts.sort_values(ascending=False)

In [ ]:
data.tail()

In [ ]:
data['unit_price'].describe()

In [ ]:
price=np.array(data['unit_price'])
price

In [ ]:
fig,ax=plt.subplots(1,1)
plt.hist(price,bins=10,color='orange')
plt.ylabel('unitPrice')
ax.legend(loc='best')
plt.show()

In [ ]:
data.loc[data['unit_price']==0].sort_values(by='quantity',ascending=False).head()

In [ ]:
data=data.loc[data['unit_price']>0].copy()

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(20,5))
sns.distplot(data['unit_price'],ax=ax[0],kde=False,color='red')
sns.distplot(np.log(data['unit_price']),ax=ax[1],bins=20,color='tomato',kde=False)
ax[1].set_xlabel('log-unit-price')

In [ ]:
data['quantity'].describe()

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(20,5))
sns.distplot(np.log(data['quantity']),ax=ax,bins=20,kde=False,color='limegreen')
ax.set_title('log_quantity distribution')
ax.set_label('Natural-Log quantity')

In [ ]:
#foucs on daily product sales
data['Revenue']=data['quantity']*data['unit_price']
data['Year']=data['invoice_date'].dt.year
data['Quarter']=data['invoice_date'].dt.quarter
data['Month']=data['invoice_date'].dt.month
data['Week']=data['invoice_date'].dt.week
data['Weekday']=data['invoice_date'].dt.weekday
data['Day']=data['invoice_date'].dt.day
data['Date']=pd.to_datetime(data[['Year','Month','Day']])


In [ ]:
grouped_features=['Date','Year','Quarter','Month','Week','Weekday','Day',
                'stock_code']

In [ ]:
daily_data=pd.DataFrame(data.groupby(grouped_features)['quantity'].sum(),columns=['quantity'])
daily_data['Revenue']=data.groupby(grouped_features)['Revenue'].sum()
daily_data=daily_data.reset_index()
daily_data.head()

In [ ]:
daily_data.loc[:,['quantity','Revenue']].describe()

In [ ]:
low_quantity=daily_data['quantity'].quantile(0.01)
high_quantity=daily_data['quantity'].quantile(0.99)
print(low_quantity,high_quantity)

In [ ]:
low_revenue=daily_data['Revenue'].quantile(0.01)
high_revenue=daily_data['Revenue'].quantile(0.99)
print(low_revenue,high_revenue)

In [ ]:
daily_data=daily_data.loc[(daily_data['quantity']>=low_quantity)&(daily_data['quantity']<=high_quantity)]
daily_data=daily_data.loc[(daily_data['Revenue']>=low_revenue)&(daily_data['Revenue']<=high_revenue)]

In [ ]:
daily_data.shape

In [ ]:
daily_data.head()

In [ ]:
week=daily_data['Week'].max()-2
print('Validation after week {}'.format(week))
print('Validation starts at timepoint {}'.format(daily_data[daily_data['Week']==week].Date.min()))


In [ ]:
X=daily_data.drop(['quantity','Revenue','Date'],axis=1)
daily_data['quantity']=np.log(daily_data['quantity'])
y=daily_data['quantity']

In [ ]:
y.plot(kind='hist')

In [ ]:
daily_data.head()

In [ ]:
# create product types
products=pd.DataFrame(index=data.loc[data['Week']<week]['stock_code'].unique(),columns=['MedianPrice'])
products['MedianPrice']=data.loc[data['Week']<week].groupby('stock_code')['unit_price'].median()
products.head()